In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Define the small CNN model
class SmallCNN(nn.Module):
    def __init__(self):
        super(SmallCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(64 * 8 * 8, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 100)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "mps"

# Load CIFAR-100 dataset
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                         download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                        download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

# Initialize the model
model = SmallCNN().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 200

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:
            print(f'Epoch [{epoch+1}/{num_epochs}], '
                  f'Step [{i+1}/{len(trainloader)}], '
                  f'Loss: {running_loss / 200:.4f}')
            running_loss = 0.0

print("Training finished!")

# Test the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the 10000 test images: {100 * correct / total}%')


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/200], Step [200/391], Loss: 4.6028
Epoch [2/200], Step [200/391], Loss: 4.5805
Epoch [3/200], Step [200/391], Loss: 4.4705
Epoch [4/200], Step [200/391], Loss: 4.1714
Epoch [5/200], Step [200/391], Loss: 3.9977
Epoch [6/200], Step [200/391], Loss: 3.8996
Epoch [7/200], Step [200/391], Loss: 3.8208
Epoch [8/200], Step [200/391], Loss: 3.7404
Epoch [9/200], Step [200/391], Loss: 3.6654
Epoch [10/200], Step [200/391], Loss: 3.6002
Epoch [11/200], Step [200/391], Loss: 3.5355
Epoch [12/200], Step [200/391], Loss: 3.4888
Epoch [13/200], Step [200/391], Loss: 3.4349
Epoch [14/200], Step [200/391], Loss: 3.4016
Epoch [15/200], Step [200/391], Loss: 3.3482
Epoch [16/200], Step [200/391], Loss: 3.2942
Epoch [17/200], Step [200/391], Loss: 3.2501
Epoch [18/200], Step [200/391], Loss: 3.2053
Epoch [19/200], Step [200/391], Loss: 3.1667
Epoch [20/200], Step [200/391], Loss: 3.1326
Epoch [21/200], Step [200/391], L

In [2]:
# save full model with weights
torch.save(model, "cifar100_full.pt", )


Model saved!
